In [103]:
import os, re, string


def pulizia(s):
    s = s.replace("\n", "")
    s = s.replace("a'", "à")
    s = s.replace("e'", "è")
    s = s.replace("i'", "ì")
    s = s.replace("o'", "ò")
    s = s.replace("u'", "ù")
    s = re.sub(r'(?<=[.,?!\'])(?=[^\s])', r' ', s)
    s = s.translate(str.maketrans('', '', string.punctuation))
    return s


d = {}
for filename in os.listdir('./Corpus/'):
    if filename.endswith('.txt'):
        with open(os.path.join('./Corpus/', filename)) as f:
            
            content = f.read()
            normalized = content.lower()
            normalized = pulizia(normalized)
            
            filename = filename.split(".")[0]
            year, pres = (filename.split("-"))
            year = int(year)
            
        d[year] = {"presidente":pres, "lunghezza":len(content.split()), "discorso_norm":normalized, "discorso_orig":content}

In [104]:
from collections import Counter
import nltk
from nltk.corpus import stopwords

ciampi2 = d[2009]['discorso_norm']
ignored_words = nltk.corpus.stopwords.words('italian')

for year, diz in d.items():
    tokens = [w for w in diz["discorso_norm"].split() if w not in ignored_words]
    d[year]["most_common"] = {k:v for k,v in Counter(tokens).most_common(30)}

In [105]:
import spacy
nlp = spacy.load('it_core_news_sm')

for year, diz in d.items():
    
    speech = nlp(diz['discorso_orig'])
    tmp = []
    
    for token in speech:         
        tmp.append(token.pos_)
    
    d[year]["PoS_percentage"] = {k:(v/d[year]["lunghezza"])*100 for k,v in dict(Counter(tmp)).items()}
    

In [106]:
import json
with open('corpus.json', 'w') as f:
    json.dump(d, f)

In [107]:
# todos
# pos tagger inconsistente durante gli anni: verificare
# output anche in csv x migliore leggibilita
# trattini e punteggiatura speciale: verificare